1. Przygotowanie danych
● Załaduj dane do środowiska Pythona.
● Zapoznaj się z dokumentacją i opisem zbioru danych.
● Połącz tabele w celu uzyskania pełnego obrazu klientów i ich pożyczek (np. połącz
dane o pożyczkach z transakcjami i rachunkami klientów).
● Utwórz zmienną opisywaną: niespłacenie pożyczki (1 = niespłacona, 0 =
spłacona).
2. Eksploracyjna analiza danych (EDA)
● Sprawdź podstawowe statystyki opisowe.
● Przeanalizuj rozkład zmiennych (histogramy, wykresy pudełkowe).
● Zidentyfikuj i obsłuż brakujące wartości.
● Sprawdź korelacje między zmiennymi a niespłaceniem pożyczki.
● Zwizualizuj zależności między cechami klientów a niespłaceniem kredytu (np. wpływ
dochodów, wieku, liczby transakcji na prawdopodobieństwo niespłaty).
3. Budowa modelu predykcyjnego
● Przygotuj dane do modelowania:
● Zakoduj zmienne kategoryczne.
● Znormalizuj lub standaryzuj dane.
● Podziel zbiór danych na zestaw treningowy i testowy.
● Zbuduj model klasyfikacyjny przewidujący, czy pożyczka nie zostanie spłacona:
● Wypróbuj różne algorytmy, np. regresję logistyczną, drzewa decyzyjne, lasy losowe,
XGBoost.
● Porównaj wyniki modeli na podstawie metryk takich jak dokładność, precyzja, recall,
F1-score (szczególnie istotny będzie *recall dla klasy "niespłacona pożyczka"*).
● Zwizualizuj krzywe ROC i macierz pomyłek.
4. Analiza wyników i wnioski
● Które cechy są najważniejsze dla przewidywania niespłacenia pożyczki?
● Jakie czynniki najczęściej wpływają na to, że klient nie spłaca kredytu?

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve

df_account = pd.read_csv('C:\Users\annal\Desktop\Junior Data Analyst\Projekt końcowy\account.csv', sep=';', header=0, index_col=0)



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1584408939.py, line 14)